# MPG Cars

### Introduction:

The following exercise utilizes data from [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Auto+MPG)

### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests
import numpy
from pyspark.sql import functions as F
import random
from pyspark import SparkContext

In [16]:
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder.master("local[1]").appName("cars").getOrCreate()

22/10/11 00:11:29 WARN Utils: Your hostname, xkeyscore resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlp0s20f3)
22/10/11 00:11:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 00:11:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/11 00:11:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sc = spark.sparkContext

### Step 2. Import the first dataset [cars1](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/05_Merge/Auto_MPG/cars1.csv) and [cars2](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/05_Merge/Auto_MPG/cars2.csv).  

In [4]:
cars_1 = spark.read.options(header=True, inferSchema=True).csv("cars1.csv")
cars_2 = spark.read.options(header=True, inferSchema=True).csv("cars2.csv")

   ### Step 3. Assign each to a variable called cars1 and cars2

### Step 4. Oops, it seems our first dataset has some unnamed blank columns, fix cars1

In [5]:
cars_1.columns

['mpg',
 'cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'model',
 'origin',
 'car',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13']

In [6]:
cars_1 = cars_1.drop('_c9', '_c10', '_c11', '_c12','_c13')

In [7]:
cars_1.show()

+----+---------+------------+----------+------+------------+-----+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin|                 car|
+----+---------+------------+----------+------+------------+-----+------+--------------------+
|18.0|        8|         307|       130|  3504|        12.0|   70|     1|chevrolet chevell...|
|15.0|        8|         350|       165|  3693|        11.5|   70|     1|   buick skylark 320|
|18.0|        8|         318|       150|  3436|        11.0|   70|     1|  plymouth satellite|
|16.0|        8|         304|       150|  3433|        12.0|   70|     1|       amc rebel sst|
|17.0|        8|         302|       140|  3449|        10.5|   70|     1|         ford torino|
|15.0|        8|         429|       198|  4341|        10.0|   70|     1|    ford galaxie 500|
|14.0|        8|         454|       220|  4354|         9.0|   70|     1|    chevrolet impala|
|14.0|        8|         440|       215|  4312|   

### Step 5. What is the number of observations in each dataset?

In [8]:
print("Count of cars1 dataset: {}".format(cars_1.count()))
print("Count of cars2 dataset: {}".format(cars_2.count()))

Count of cars1 dataset: 198
Count of cars2 dataset: 200


### Step 6. Join cars1 and cars2 into a single DataFrame called cars

In [9]:
print(cars_1.columns)
print(cars_2.columns)

['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model', 'origin', 'car']
['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model', 'origin', 'car']


In [29]:
total_cars = cars_1.union(cars_2)

In [30]:
total_cars.count()

398

### Step 7. Oops, there is a column missing, called owners. Create a random number Series from 15,000 to 73,000.

In [35]:
total_cars = total_cars.withColumn("owners",F.row_number().over(Window.orderBy('mpg'))+14999)

In [36]:
total_cars.tail(2)

22/10/11 00:18:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/11 00:18:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/11 00:18:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/11 00:18:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/11 00:18:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


[Row(mpg=44.6, cylinders=4, displacement=91, horsepower='67', weight=1850, acceleration=13.8, model=80, origin=3, car='honda civic 1500 gl', number=15396, owners=15396),
 Row(mpg=46.6, cylinders=4, displacement=86, horsepower='65', weight=2110, acceleration=17.9, model=80, origin=3, car='mazda glc', number=15397, owners=15397)]

In [37]:
total_cars.show()

22/10/11 00:18:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/11 00:18:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/11 00:18:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+----+---------+------------+----------+------+------------+-----+------+--------------------+------+------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin|                 car|number|owners|
+----+---------+------------+----------+------+------------+-----+------+--------------------+------+------+
| 9.0|        8|         304|       193|  4732|        18.5|   70|     1|            hi 1200d| 15000| 15000|
|10.0|        8|         360|       215|  4615|        14.0|   70|   

### Step 8. Add the column owners to cars